<a href="https://colab.research.google.com/github/BATiger/Classic/blob/main/Lenet_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
# average pooling
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5),
            nn.AvgPool2d(kernel_size=2),           
        )
        self.classifier = nn.Sequential(
            nn.Linear(400,120),
            nn.Linear(120,84),
            nn.Linear(84,10),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x

In [ ]:
batch_size = 32
epochs = 20
learning_rate = 0.0002
GPU = True

# choose device
print(torch.cuda.is_available())
if GPU:
    device = torch.device("cuda")

True


In [ ]:
transform = transforms.Compose([
          transforms.ToTensor()
])

trainset = datasets.CIFAR10(root='data', train=True, download=True,transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = datasets.CIFAR10(root='data',train=False,download=True,transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size,shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = LeNet().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    count = 0
    print('epoch {}'.format(epoch + 1))
    # training------------------------------------------
    train_loss = 0
    train_acc = 0
    for x,y in train_loader:
        x,y = x.to(device), y.to(device)
        output = model(x)
        loss = criterion(output, y)

        train_loss += loss.item()
        # torch.max 返回 (1)最大值 (2)最大值的index 这里要的是index
        pred = torch.max(output,1)[1]
        
        train_correct = (pred == y).sum()
        train_acc += train_correct.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count = count+1

    # 打印每一个epoch的结果
    print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / count, train_acc / (len(trainset)))) 

epoch 1
Train Loss: 1.924096, Acc: 0.312860
epoch 2
Train Loss: 1.805974, Acc: 0.369400
epoch 3
Train Loss: 1.777875, Acc: 0.382700
epoch 4
Train Loss: 1.762956, Acc: 0.390860
epoch 5
Train Loss: 1.750757, Acc: 0.395700
epoch 6
Train Loss: 1.744008, Acc: 0.399100
epoch 7
Train Loss: 1.739175, Acc: 0.403260
epoch 8
Train Loss: 1.733063, Acc: 0.408000
epoch 9
Train Loss: 1.729764, Acc: 0.409100
epoch 10
Train Loss: 1.727329, Acc: 0.408560
epoch 11
Train Loss: 1.725031, Acc: 0.410140
epoch 12
Train Loss: 1.722090, Acc: 0.413300
epoch 13
Train Loss: 1.720913, Acc: 0.412500
epoch 14
Train Loss: 1.718369, Acc: 0.414440
epoch 15
Train Loss: 1.717522, Acc: 0.415120
epoch 16
Train Loss: 1.715857, Acc: 0.415600
epoch 17
Train Loss: 1.713880, Acc: 0.415080
epoch 18
Train Loss: 1.713001, Acc: 0.416180
epoch 19
Train Loss: 1.711563, Acc: 0.416840
epoch 20
Train Loss: 1.710439, Acc: 0.417280


In [ ]:
with torch.no_grad():
    model.eval()
    eval_loss = 0.
    eval_acc = 0.

    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss =  criterion(out, y)
        eval_loss += loss.item()
        pred = torch.max(out, 1)[1]
        num_correct = (pred == y).sum()
        eval_acc += num_correct.item()
    
    print('test_loss: ', eval_loss/len(test_loader))
    print('test accuracy: ', eval_acc/len(testset))
    #0.40

test_loss:  1.7165622539794483
test accuracy:  0.4122


结果发现：
1.需要尽量小的learning rate
2.Lenet能力并不足以处理RGB图像
3.小的batch_size可能更好（我猜测是因为class的数量比较小